In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import argparse
import os
import time
from Network import BrainNet
from Loss import *
from NeuralODE import *
from Utils import *
from Registration import registration, evaluation_anova

In [3]:
parser = argparse.ArgumentParser()
# File path
parser.add_argument("--savepath", type=str,
                    dest="savepath", default='./result',
                    help="path for saving results")
parser.add_argument("--fixed", type=str,
                    dest="fixed", default='./data/OAS1_0001_MR1/brain.nii.gz',
                    help="fixed image data path")
parser.add_argument("--moving", type=str,
                    dest="moving", default='./data/OAS1_0002_MR1/brain.nii.gz',
                    help="moving image data path")
parser.add_argument("--fixed_seg", type=str,
                    dest="fixed_seg", default='./data/OAS1_0001_MR1/brain_aseg.nii.gz',
                    help="fixed image segmentation data path")
parser.add_argument("--moving_seg", type=str,
                    dest="moving_seg", default='./data/OAS1_0002_MR1/brain_aseg.nii.gz',
                    help="moving image segmentation data path")
# Model configuration
parser.add_argument("--ds", type=int,
                    dest="ds", default=2,
                    help="specify output downsample times.")
parser.add_argument("--bs", type=int,
                    dest="bs", default=16,
                    help="bottleneck size.")
parser.add_argument("--smoothing_kernel", type=str,
                    dest="smoothing_kernel", default='AK',
                    help="AK: Averaging kernel; GK: Gaussian Kernel")
parser.add_argument("--smoothing_win", type=int,
                    dest="smoothing_win", default=15,
                    help="Smoothing Kernel size")
parser.add_argument("--smoothing_pass", type=int,
                    dest="smoothing_pass", default=1,
                    help="Number of Smoothing pass")
# Training configuration
parser.add_argument("--time_steps", type=int,
                    dest="time_steps", default=2,
                    help="number of time steps between the two images, >=2.")
parser.add_argument("--optimizer", type=str,
                    dest="optimizer", default='Euler',
                    help="Euler or RK.")
parser.add_argument("--STEP_SIZE", type=float,
                    dest="STEP_SIZE", default=0.001,
                    help="step size for numerical integration.")
parser.add_argument("--epoches", type=int,
                    dest="epoches", default=300,
                    help="No. of epochs to train.")
parser.add_argument("--NCC_win", type=int,
                    dest="NCC_win", default=21,
                    help="NCC window size")
parser.add_argument("--lr", type=float,
                    dest="lr", default=0.005,
                    help="Learning rate.")
parser.add_argument("--lambda_J", type=int,
                    dest="lambda_J", default=2.5,
                    help="Loss weight for neg J")
parser.add_argument("--lambda_df", type=int,
                    dest="lambda_df", default=0.05,
                    help="Loss weight for dphi/dx")
parser.add_argument("--lambda_v", type=int,
                    dest="lambda_v", default=0.00005,
                    help="Loss weight for neg J")
parser.add_argument("--loss_sim", type=str,
                    dest="loss_sim", default='NCC',
                    help="Similarity measurement")
# Debug
parser.add_argument("--debug", type=bool,
                    dest="debug", default=False,
                    help="debug mode")
# Device
parser.add_argument("--device", type=str,
                    dest="device", default='cuda:0',
                    help="gpu: cuda:0; cpu: cpu")

parser.add_argument("--f", type=str,
                    dest='not_important')

config, unknown = parser.parse_known_args()
if not os.path.isdir(config.savepath):
    os.makedirs(config.savepath)

In [5]:
dice_data = {
    'epochs':[],
    'learning_rate':[],
    'smoothing_kernel':[],
    'file':[],
    'value':[]
}

jacob_data = {
    'epochs':[],
    'learning_rate':[],
    'smoothing_kernel':[],
    'file':[],
    'value':[]
}

epoches = [5,10,20]
learning_rates = [0.001, 0.005, 0.01]
kernels = ['AK','GK']
files = [
            (
                'data (1)/OASIS_OAS1_0001_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0001_MR1/aligned_seg35.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_seg35.nii.gz'
            ),
            (
                'data (1)/OASIS_OAS1_0001_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0001_MR1/aligned_seg35.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_seg35.nii.gz'
            )
      ]


config.device = 'cpu'

for kernel in kernels:
    config.smoothing_kernel = kernel
    for lr in learning_rates:
        config.lr = lr
        for epoch in epoches:
            config.epoches = epoch
            for fixed, moving, fixed_seg, moving_seg in files:
                device = torch.device(config.device)
                fixed = load_nii(config.fixed)
                moving = load_nii(config.moving)
                assert fixed.shape == moving.shape  # two images to be registered must in the same size
                df, df_with_grid, warped_moving = registration(config, device, moving, fixed)
                mean_dice, std_dice, ratio_neg_J = evaluation_anova(config, device, df, df_with_grid)

                print(f'Done with running:\n\tkernel: {kernel}\n\tlearning_rate: {lr}\n\tepoches: {epoch}')

                dice_data['epochs'].append(epoch)
                dice_data['learning_rate'].append(lr)
                dice_data['smoothing_kernel'].append(kernel)
                dice_data['file'].append(f'fixed: {fixed}, moving: {moving}')
                dice_data['value'].append(mean_dice)

                jacob_data['epochs'].append(epoch)
                jacob_data['learning_rate'].append(lr)
                jacob_data['smoothing_kernel'].append(kernel)
                jacob_data['file'].append(f'fixed: {fixed}, moving: {moving}')
                jacob_data['value'].append(ratio_neg_J)

pd.DataFrame(dice_data).to_csv('dice_data.csv')
pd.DataFrame(jacob_data).to_csv('jacob_data.csv')


Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate: 0.001
	epoches: 5
Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate: 0.001
	epoches: 5
Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate: 0.001
	epoches: 10
Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate: 0.001
	epoches: 10
Iteration: 20 Loss_sim: 6.426e-01 loss_J: 3.612e-04
Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate: 0.001
	epoches: 20
Iteration: 20 Loss_sim: 6.402e-01 loss_J: 3.264e-03
Total of neg Jet:  23.956266
Ratio of neg Jet:  9.625199786219246e-05
Done with running:
	kernel: AK
	learning_rate: 0.001
	epoches: 20
Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate: 0.005
	epoches: 5
Total of neg Jet:  0.0
Ratio of neg Jet:  0.0
Done with running:
	kernel: AK
	learning_rate

In [32]:
dice_data = pd.read_csv('dice_data.csv').drop(columns='file')
jacob_data = pd.read_csv('jacob_data.csv').drop(columns='file')

dice_data 

,Unnamed: 0,epochs,learning_rate,smoothing_kernel,value
0,0,5,0.001,AK,0.509185
1,1,5,0.001,AK,0.509185
2,2,10,0.001,AK,0.506616
3,3,10,0.001,AK,0.518514
4,4,20,0.001,AK,0.546937
5,5,20,0.001,AK,0.546231
6,6,5,0.005,AK,0.509185
7,7,5,0.005,AK,0.509185
8,8,10,0.005,AK,0.509185
9,9,10,0.005,AK,0.556740


In [29]:
def ANOVA(experiments, reps, accuracies):
    levels = [len(experiments[k]) for k in experiments.keys()]

    total_mean = np.mean(accuracies)

    ssa = levels[1] * levels[2] * reps * np.sum([(np.mean(accuracies[a,:,:,:]) - total_mean)**2 for a in range(levels[0])])
    ssb = levels[0] * levels[2] * reps * np.sum([(np.mean(accuracies[:,b,:,:]) - total_mean)**2 for b in range(levels[1])])
    ssc = levels[0] * levels[1] * reps * np.sum([(np.mean(accuracies[:,:,c,:]) - total_mean)**2 for c in range(levels[2])])

    ssab = levels[2] * reps * np.sum([[(np.mean(accuracies[a,b,:,:]) - np.mean(accuracies[a,:,:,:]) - np.mean(accuracies[:,b,:,:])\
                               + total_mean)**2 for a in range(levels[0])] for b in range(levels[1])])
    ssac = levels[1] * reps * np.sum([[(np.mean(accuracies[a,:,c,:]) - np.mean(accuracies[a,:,:,:]) - np.mean(accuracies[:,:,c,:])\
                               + total_mean)**2   for a in range(levels[0])] for c in range(levels[2])])
    ssbc = levels[0] * reps * np.sum([[(np.mean(accuracies[:,b,c,:]) - np.mean(accuracies[:,b,:,:]) - np.mean(accuracies[:,:,c,:])\
                               + total_mean)**2   for b in range(levels[1])] for c in range(levels[2])])

    ssabc = reps * np.sum([[[(np.mean(accuracies[a,b,c,:]) - np.mean(accuracies[a,b,:,:]) - np.mean(accuracies[:,b,c,:])\
                                - np.mean(accuracies[a,:,c,:]) + np.mean(accuracies[a,:,:,:]) + np.mean(accuracies[:,b,:,:])\
                                + np.mean(accuracies[:,:,c,:]) - total_mean)**2 for a in range(levels[0])]\
                                for b in range(levels[1])] for c in range(levels[2])])

    ssy = np.sum(accuracies**2)
    ss0 = levels[0] * levels[1] * levels[2]* reps * total_mean**2
    sst = ssy - ss0
    sse = sst - ssa - ssb - ssab - ssac - ssbc - ssabc

    l = levels
    r = reps
    ss = np.array([ssa,      ssb,      ssc,      ssab,             ssac,             ssbc,             ssabc, sse])
    dof = np.array([(l[0]-1), (l[1]-1), (l[2]-1), (l[0]-1)*(l[1]-1), (l[0]-1)*(l[2]-1), (l[1]-1)*(l[2]-1),\
                    (l[0]-1)*(l[1]-1)*(l[2]-1), l[0]*l[1]*l[2]*(r-1)])

    ms = ss/dof
    f_comp = ms/ms[-1]

    import scipy.stats
    f_table = [scipy.stats.f.ppf(q=1-.05, dfn=dof[i], dfd=dof[-1]) for i in range(len(dof)-1)] + [None]

    table = {
        'ss':ss,
        'dof':dof,
        'ms':ms,
        'F_comp':f_comp,
        'F_table':f_table
    }

    return pd.DataFrame(table, index=['A','B','C','AB','AC','BC','ABC','Error'])

In [30]:
values = np.zeros((2,3,3,2))


for i_a, kernel in enumerate(kernels):
    data_l1 = dice_data[dice_data['smoothing_kernel'] == kernel]
    for i_b, lr in enumerate(learning_rates):
        data_l2 = data_l1[data_l1['learning_rate'] == lr]
        for i_c, epoch in enumerate(epoches):
            data_l3 = data_l2[data_l2['epochs'] == epoch]['value']
            values[i_a, i_b, i_c, :] = data_l3.values
            
        
experiments = {
    'kernels':kernels,
    'learning_rates':learning_rates,
    'epoches':epoches
}

reps = 2


anova_output = ANOVA(experiments, reps, values)
anova_output

,ss,dof,ms,F_comp,F_table
A,0.000034,1,0.000034,0.015168,4.413873
B,0.005058,2,0.002529,1.135638,3.554557
C,0.025219,2,0.012610,5.661957,3.554557
AB,0.007939,2,0.003970,1.782415,3.554557
AC,0.001489,2,0.000745,0.334331,3.554557
BC,0.003861,4,0.000965,0.433366,2.927744
ABC,0.005454,4,0.001364,0.612274,2.927744
Error,0.040087,18,0.002227,1.000000,NaN


In [37]:
values = np.zeros((2,3,3,2))


for i_a, kernel in enumerate(kernels):
    data_l1 = jacob_data[jacob_data['smoothing_kernel'] == kernel]
    for i_b, lr in enumerate(learning_rates):
        data_l2 = data_l1[data_l1['learning_rate'] == lr]
        for i_c, epoch in enumerate(epoches):
            data_l3 = data_l2[data_l2['epochs'] == epoch]['value']
            values[i_a, i_b, i_c, :] = data_l3.values
            
        
experiments = {
    'kernels':kernels,
    'learning_rates':learning_rates,
    'epoches':epoches
}

reps = 2


anova_output = ANOVA(experiments, reps, 1-values)
anova_output

,ss,dof,ms,F_comp,F_table
A,0.000057,1,0.000057,0.259242,4.413873
B,0.000256,2,0.000128,0.581635,3.554557
C,0.000579,2,0.000289,1.316811,3.554557
AB,0.000548,2,0.000274,1.246425,3.554557
AC,0.000138,2,0.000069,0.314941,3.554557
BC,0.000504,4,0.000126,0.573858,2.927744
ABC,0.000948,4,0.000237,1.078730,2.927744
Error,0.003956,18,0.000220,1.000000,NaN
